In [5]:
import pandas as pd
from datetime import datetime, timedelta
import time, random

class ExecutionTimeDecorator:
    def __init__(self, function):
        self.function = function
    def __call__(self, *args, **kwargs):
        st = datetime.now().timestamp()
        res = self.function(*args, **kwargs)
        print("[%s Function Run-Time : %0.4fsec" % (self.function.__name__, datetime.now().timestamp()-st))
        return res

@ExecutionTimeDecorator
def data_frame_info(_df):
    cols = list(_df.columns)
    print("Columns : ", cols)
    print("Shape : ", _df.shape)
    # print("Unique : ", data_frame.unique())
    print("Value_Counts : ", _df.value_counts(normalize=True))
    # a = data_frame['상담자'].value_counts(normalize=True)
    # print(a, "\n", type(a), "\n", list(a), "\n", dict(a))
    return None

@ExecutionTimeDecorator
def convert_from_excel(_file_name):
    data_frame = pd.read_excel(_file_name, converters=RAW_DATA_TYPE)
    data_frame = data_frame.astype(str)
    time.sleep(0.1)
    return data_frame

def create_datetime(str_date, end_date):
    str_obj = datetime.strptime(str_date, '%Y-%m-%d %H:%M:%S')
    end_obj = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    # print(str_obj, end_obj)
    def random_date(_str, _end):
        return _str + timedelta(seconds=random.randint(0, int((_end - _str).total_seconds())))
    return str(random_date(str_obj, end_obj))

@ExecutionTimeDecorator
def create_dict_case(_dict, loop):
    if type(_dict) is not dict:
        return "Datatype is not Dictionary !"    
    params = list(_dict.keys())
    weights = list(_dict.values())
    # print(_dict.keys())
    # print(_dict.values())
    res = random.choices(params, weights=weights, k=loop)
    # print(pd.DataFrame(res).value_counts(normalize=True))
    return res

# Not use
def create_args_case(*args):
    print(type(args))
    if len(args) % 2 != 0:
        return "ㅜㅜ 짝수가 아냐"
    else:
        params = args[::2]  # offset
        weights = args[1::2]
        print(params, weights)
        print(random.choices(params, weights=weights, k=2))
        return "짝수양"
# print(create_args_case("a", 1, "b", 2, "c", 3))

@ExecutionTimeDecorator
def multiple_create_file(_df, cnt, st_list, ed_list):
    if len(st_list) != len(ed_list) and len(st_list) != cnt:
        return False
    loop_cnt = 0
    while cnt > loop_cnt:
        _df = create_columns_data(_df)
        print(st_list[loop_cnt], ed_list[loop_cnt])
        _df = crate_date_and_index_data(_df, st_list[loop_cnt], ed_list[loop_cnt])
        # Excel Export 전에 Columns 배치 순서 및 정렬 
        _df = _df[COLUMNS_INDEX]
        _df.sort_index(axis=0)
        print("\n", _df)
        _df.to_excel(DESKTOP_PATH + CREATE_FILE_NAME + str(loop_cnt) + ".xlsx", 
                     sheet_name='Sheet1', index=False)
        loop_cnt += 1
        time.sleep(0.1)
    return True

@ExecutionTimeDecorator
def create_columns_data(_df):
    columns = list(_df.columns)
    for col in columns:
        r = create_dict_case(dict(raw[col].value_counts(normalize=True)), ROW_NUMBER)
        _df[col] = r
    return _df

@ExecutionTimeDecorator
def crate_date_and_index_data(_df, st, ed):
    # todo : 날짜와 Index 생성
    _date = []
    _index = []
    for i in range(0, ROW_NUMBER):
        _date.append(create_datetime(st, ed))
        _index.append(i)
    _date.sort()
    _df[DATE_FIELD_NAME] = _date
    _df[INDEX_FIELD_NAME] = _index
    return _df

""" Raw Data Unique Count per Column 
    번호 : index 번호 
    상담일 : 2020-03-20 13:35:13 형식
    인/아웃 : in,out 2가지
    상담유형 : 8개
    상담방법 : 1개
    상담결과 : 1개
    대표번호 : 2개
    IVR메뉴 : 5개
    상담자 : 6개
    상담내용 : 12386개
    고장증상 : 7개
    코드 : 7개
    고장원인 : 50개
    코드.1 : 2개
    조치사항 : 10개
    코드.2 : 10개
    처리여부 : 2개
"""

""" loc[index] 새로운 list를 새로운 row로 가져와 index에 추가하는 함수 
    append(dictionary)는 dict 값을 행으로 직접 가져와 추가하는 함수, but ignore_index = True 이어야 함 
    Example : df.loc[1] = ['1','2','3']
    Example : df = df.append({'1':1}, ignore_index=True) """

# todo : (Warning) MULTI_FILE num == Length of start_date_str_list and end_date_str_list
DESKTOP_PATH = "C:/Users/CheonYoungJo/Desktop/"
CREATE_FILE_NAME = "test"
RAW_DATA_TYPE = {'대표번호': str}
COLUMNS_INDEX = ['번호', '상담일', '인/아웃', '상담유형', '상담방법', '상담결과', '대표번호', 'IVR메뉴', '상담자', '상담내용', '고장증상', '코드', '고장원인', '코드.1', '조치사항', '코드.2', '처리여부']
start_date_str_list = ['2020-01-01 00:00:00', '2020-02-01 00:00:00', '2020-03-01 00:00:00']
end_date_str_list = ['2020-01-31 23:59:59', '2020-02-29 23:59:59', '2020-03-31 23:59:59']
#file_name = "상담및 수리 이력(20년 1월_4월)_공유용.xlsx"
file_name = "상담및 수리 이력(20년 5월_8월)_공유용.xlsx"
ROW_NUMBER = 10000
MULTI_FILE = 3
INDEX_FIELD_NAME = '번호'
DATE_FIELD_NAME = '상담일'
remove_fields = [INDEX_FIELD_NAME, DATE_FIELD_NAME]

raw = convert_from_excel(file_name)
data_frame_info(raw)
df = pd.DataFrame(columns=list(raw.columns))

# print(create_dict_case(dict(raw['상담자'].value_counts(normalize=True)), 1000))
# todo : 원본의 특정 필드 제거 후 나머지 필드 모두 더미 생성
for field in remove_fields:
    del df[field]

multiple_create_file(df, MULTI_FILE, start_date_str_list, end_date_str_list)

[convert_from_excel Function Run-Time : 4.5095sec
Columns :  ['번호', '상담일', '인/아웃', '상담유형', '상담방법', '상담결과', '대표번호', 'IVR메뉴', '상담자', '상담내용', '고장증상', '코드', '고장원인', '코드.1', '조치사항', '코드.2', '처리여부']
Shape :  (21723, 17)
Value_Counts :  번호     상담일                  인/아웃  상담유형        상담방법  상담결과  대표번호         IVR메뉴      상담자  상담내용                                                                                                     고장증상  코드  고장원인  코드.1  조치사항  코드.2  처리여부
9999   2020-06-29 11:25:21  In    nan         전화    성공    nan          nan        현진영  충전후 잠시 동작했다가 노랑불로 바뀜/700\r\n홈페이지접수/부재                                                                    고장증상  0   고장원인  0.0   조치사항  0     미완료     0.000046
16517  2020-08-05 14:28:11  Out   nan         전화    성공    07049387920  nan        김민정  3i / 제품은 작동되는데 앱이 안된다. 정보삭제후 재등록안내. 노인이라 자세히 설명해달라. 링크전송                                                 고장증상  0   고장원인  0.0   조치사항  0     미완료     0.000046
1651   2020-05-14 13:58:54  In    nan         전화    성

True

In [130]:
import datetime
from pytz import timezone
fmt = '%Y-%m-%dT%H:%M:%S'
UTC = timezone('UTC')
KST = timezone('Asia/Seoul')
date = '2020-10-14T00:00:00'
udate = UTC.localize(datetime.datetime.strptime(date, fmt))
kdate = KST.localize(datetime.datetime.strptime(date, fmt))

print("KST Date : %s \nUTC Date : %s" % (kdate, udate))
print("KST to UTC : %s" % str(kdate.astimezone(UTC)))

KST Date : 2020-10-14 00:00:00+09:00 
UTC Date : 2020-10-14 00:00:00+00:00
KST to UTC : 2020-10-13 15:00:00+00:00
